In [8]:
from PIL import Image
import numpy as np

# Load an image from the file system
image_path = '/home/henry/robo/cis5810/final/gcs_images/gavd_frame_images/cljaqyc4800az3n6l3qio6zbh/cljaqyc4800az3n6l3qio6zbh_frame_811.jpg'
image = Image.open(image_path)
# convert image to numpy array
image.show()
image = np.array(image)


In [37]:
import mediapipe as mp
import cv2

# Initialize MediaPipe Pose Landmark Detector    
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)
# Process the image
results = pose.process(image)
# plot the landmarks
image_landmarks = image.copy()
# make image much bigger
image_landmarks = cv2.resize(image_landmarks, (image_landmarks.shape[1]*4, image_landmarks.shape[0]*4))
print(dir(results.pose_landmarks.landmark[0]))

points = [[int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0]), int(100*landmark.visibility)] for landmark in results.pose_landmarks.landmark]
points = np.array(points)
x = points[:,0]
y = points[:,1]
presence = points[:,2]
min_x = np.min(x)
max_x = np.max(x)
min_y = np.min(y)-10
max_y = np.max(y)
print(points)

# Draw landmarks labeled with presence
for i, landmark in enumerate(results.pose_landmarks.landmark):
    cv2.circle(image_landmarks, (int(landmark.x * image_landmarks.shape[1]), int(landmark.y * image_landmarks.shape[0])), 1, (0, 255, 0), -1)
    cv2.putText(image_landmarks, str(i), (int(landmark.x * image_landmarks.shape[1]), int(landmark.y * image_landmarks.shape[0])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(image_landmarks, str(presence[i]), (int(landmark.x * image_landmarks.shape[1]), int(landmark.y * image_landmarks.shape[0])-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

# Draw bounding box around subject

image_landmarks = cv2.rectangle(image_landmarks, (min_x, min_y), (max_x, max_y), (0, 255, 0), 2)
# Display the image with landmarks
image_landmarks = Image.fromarray(image_landmarks)
image_landmarks.show()

I0000 00:00:1732995455.345402  131223 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1732995455.346614  140324 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 21.2.6), renderer: Mesa Intel(R) Graphics (ADL GT2)


['ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'Extensions', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'MergeFrom', 'MergeFromString', 'ParseFromString', 'RegisterExtension', 'SerializePartialToString', 'SerializeToString', 'SetInParent', 'UnknownFields', 'WhichOneof', '_CheckCalledFromGeneratedFile', '_SetListener', '__class__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__unicode__', '_extensions_by_name', '_extensions_by_number', 'presence', 'visibility', 'x', 'y', 'z']
[[125  27  99]
 [123  23  99]
 [123  23  99]
 [123  23  99]
 [121  24  99]
 [120  2

In [2]:
import pandas as pd

dataset = pd.read_pickle("/home/henry/robo/cis5810/final/updated_dataset.pkl")
# create df with unique values of [seq, cam_view, gait_pat]
df = dataset[['seq', 'cam_view', 'gait_pat']].drop_duplicates()
period = pd.read_csv("/home/henry/robo/cis5810/final/frames_per_period.csv")
period = period.rename(columns={'clip': 'seq'})
combined = pd.merge(df, period, on='seq')
print(combined.head())

                         seq cam_view  gait_pat      freq   is_0  period
0  clld3ef9b003p3o6l1f3mrlez     back  abnormal  0.847458  False      35
1  clld3fen1003v3o6l9ecbpfk9    front  abnormal  0.852273  False      35
2  cll9v50ee004y3o6lj9djtcfk     back  abnormal  0.552632  False      54
3  cll9v6n7200523o6lbcj3iixb    front  abnormal  0.592105  False      51
4  cljwssdku00093n6lv3u03u8b    front  abnormal  0.885246  False      34


In [ ]:
import os 
import cv2
import numpy as np
import imageio

image_path = '/home/henry/robo/cis5810/final/gcs_images/gavd_frame_images'
save_dir = f'/home/henry/robo/cis5810/final/demo_app/gifs'
for idx, row in combined.iterrows():
    seq = row['seq']
    cam_view = row['cam_view']
    gait_pat = row['gait_pat']
    period = row['period']
    image_dir = os.path.join(image_path, seq)
    # extract the frame number from each file in the directory
    frames = [int(f.split('_')[-1].split('.')[0]) for f in os.listdir(image_dir)]
    frames = np.sort(frames)
    
    print(f' max: {np.max(frames)} min: {np.min(frames)}')
    interval = np.ceil(period / 20).astype(int)
    print(f'interval: {interval}')

    # downsample the frames according to the period so one period is captured in 20 frames
    downsampled_frames = frames[::interval]
    downsampled_frames = downsampled_frames[:30]
    print(f'frames: {downsampled_frames}')

    # save the downsampled images
    save_dir = f'/home/henry/robo/cis5810/final/demo_app/gifs/{gait_pat}/{cam_view}'
    os.makedirs(save_dir, exist_ok=True)
    # Save the frames as a GIF
    with imageio.get_writer(os.path.join(save_dir,f"{seq}.gif"), mode="I") as writer:
        for frame in downsampled_frames:
            file = os.path.join(image_dir, f'{seq}_frame_{frame}.jpg')
            image = cv2.imread(file)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            writer.append_data(image)
    



 max: 2816 min: 2639
interval: 2
frames: [2639 2641 2643 2645 2647 2649 2651 2653 2655 2657 2659 2661 2663 2665
 2667 2669 2671 2673 2675 2677]
 max: 3028 min: 2852
interval: 2
frames: [2852 2854 2856 2858 2860 2862 2864 2866 2868 2870 2872 2874 2876 2878
 2880 2882 2884 2886 2888 2890]
 max: 3676 min: 3296
interval: 3
frames: [3296 3299 3302 3305 3308 3311 3314 3317 3320 3323 3326 3329 3332 3335
 3338 3341 3344 3347 3350 3353]
 max: 4083 min: 3779
interval: 3
frames: [3779 3782 3785 3788 3791 3794 3797 3800 3803 3806 3809 3812 3815 3818
 3821 3824 3827 3830 3833 3836]
 max: 314 min: 9
interval: 2
frames: [ 9 11 13 15 17 19 21 23 25 27 29 31 33 35 37 39 41 43 45 47]
 max: 590 min: 341
interval: 2
frames: [341 343 345 347 349 353 355 358 360 363 367 371 375 378 381 383 385 387
 390 392]
 max: 584 min: 404
interval: 2
frames: [404 406 408 410 412 414 416 418 420 422 424 426 428 430 432 434 436 438
 440 442]
 max: 908 min: 721
interval: 2
frames: [721 723 725 727 729 731 733 735 737 739 7

KeyboardInterrupt: 